# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation. To run this example Julia's `PyCall` package
needs to be able to find an installation of `pymatgen`.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

┌ Warning: `vendor()` is deprecated, use `BLAS.get_config()` and inspect the output instead
│   caller = npyinitialize() at numpy.jl:67
└ @ PyCall /home/runner/.julia/packages/PyCall/L0fLP/src/numpy.jl:67


Setup an LDA model and discretize using
a single k-point and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=(1, 1, 1))

PlaneWaveBasis discretization:
    Ecut                 : 5.0 Ha
    fft_size             : (32, 32, 32)
    kgrid type           : Monkhorst-Pack
    kgrid                : [1, 1, 1]
    num. irred. kpoints  : 1

    Discretized Model(lda_xc_teter93, 3D):
        lattice (in Bohr)    : [0         , 10.2631   , 10.2631   ]
                               [10.2631   , 0         , 10.2631   ]
                               [10.2631   , 10.2631   , 0         ]
        unit cell volume     : 2162.1 Bohr³
    
        atoms                : Si₁₆
        atom potentials      : ElementPsp(Si, psp=hgh/lda/si-q4)
    
        num. electrons       : 64
        spin polarization    : none
        temperature          : 0 Ha
    
        terms                : Kinetic()
                               AtomicLocal()
                               AtomicNonlocal()
                               Ewald()
                               PspCorrection()
                               Hartree()
            

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.124118e+02     1.512749e+00
 * time: 0.7492871284484863
     1     1.018928e+01     9.380359e-01
 * time: 2.0284860134124756
     2    -1.190689e+01     1.036276e+00
 * time: 2.636230945587158
     3    -3.427936e+01     7.103141e-01
 * time: 3.5406720638275146
     4    -4.754369e+01     5.735439e-01
 * time: 4.446378946304321
     5    -5.681562e+01     2.236991e-01
 * time: 5.405240058898926
     6    -5.960100e+01     1.675692e-01
 * time: 6.020704984664917
     7    -6.073749e+01     7.621388e-02
 * time: 6.623304128646851
     8    -6.123020e+01     4.758269e-02
 * time: 7.2215659618377686
     9    -6.151687e+01     5.073645e-02
 * time: 7.858420133590698
    10    -6.171760e+01     2.911768e-02
 * time: 8.460159063339233
    11    -6.189047e+01     2.296046e-02
 * time: 9.022659063339233
    12    -6.200197e+01     1.709002e-02
 * time: 9.566000938415527
    13    -6.205425e+01     1.529138e-02
 * time: 10.13927412033081
   

In [5]:
scfres.energies

Energy breakdown (in Ha):
    Kinetic             25.7671067
    AtomicLocal         -18.8557667
    AtomicNonlocal      14.8522637
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485368 
    Xc                  -19.3336819

    total               -62.261666461740